# 1. Preparación de los datos

In [13]:
pip install unidecode

Note: you may need to restart the kernel to use updated packages.


In [14]:
!pip install scikit-learn

In [41]:
# Importación de las librerías necesarias
import pandas as pd
import numpy as np
import re
from unidecode import unidecode
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [42]:
# Cargamos el archivo CSV
df = pd.read_csv('../datasets/items_titles.csv')

In [43]:
print("\nInformacion general del dataset")
df.info()

df


Informacion general del dataset
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ITE_ITEM_TITLE  10000 non-null  object
dtypes: object(1)
memory usage: 78.3+ KB


,ITE_ITEM_TITLE
0,Tênis Olympikus Esporte Valente - Masculino Kids
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...
2,Tênis Usthemp Slip-on Temático - Labrador 2
3,Tênis Casual Feminino Moleca Tecido Tie Dye
4,Tênis Star Baby Sapatinho Conforto + Brinde
...,...
9995,Chuteira Futsal Oxn Velox 3 Infantil
9996,Sapatenis Casual Masculino Estiloso 24horas Co...
9997,Tênis Feminino Infantil Molekinha Tie Dye
9998,Tênis Feminino Leve Barato Ganhe 1 Colchonete ...


Podemos observar que el dataset contiene una columna llamada `ITE_ITEM_TITLE` con los títulos de productos, que vamos a usar para medir similitud textual.

In [47]:
df_clean = df.copy()

# Función de limpieza de texto actualizada
def clean_text(text):
    if pd.isnull(text):
        return ""
    text = text.lower()  # Pasar a minúsculas
    text = unidecode(text)  # Eliminar acentos (tênis -> tenis)
    text = re.sub(r'[^\w\s]', '', text)  # Eliminar signos de puntuación
    text = re.sub(r'\s+', ' ', text).strip()  # Eliminar espacios duplicados
    return text

# Aplicamos la función al dataset
df_clean['clean_title'] = df_clean['ITE_ITEM_TITLE'].apply(clean_text)

#Mostramos una muestra de antes y después
df_clean[['ITE_ITEM_TITLE', 'clean_title']].head(10)

,ITE_ITEM_TITLE,clean_title
0,Tênis Olympikus Esporte Valente - Masculino Kids,tenis olympikus esporte valente masculino kids
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...,bicicleta barra forte samy c 6 marchas cubo c ...
2,Tênis Usthemp Slip-on Temático - Labrador 2,tenis usthemp slipon tematico labrador 2
3,Tênis Casual Feminino Moleca Tecido Tie Dye,tenis casual feminino moleca tecido tie dye
4,Tênis Star Baby Sapatinho Conforto + Brinde,tenis star baby sapatinho conforto brinde
5,Tênis Oakley Frequency 3.0 Preto/marrom,tenis oakley frequency 30 pretomarrom
6,Tênis Jogging Feminino Premium Super Lançament...,tenis jogging feminino premium super lancament...
7,Under Armour Hovr Phantom 2 Conexão Bluetooth ...,under armour hovr phantom 2 conexao bluetooth ...
8,Tenis Infantil Feminino Menina Criança Moça,tenis infantil feminino menina crianca moca
9,Tênis Labellamafia Saturn 6 Cores Disponíveis,tenis labellamafia saturn 6 cores disponiveis


Relicé una limpieza adicional (normalización) eliminando los acentos y caracteres especiales utilizando la librería unidecode. Ya que los títulos están en portugués (ej: "tênis", "camiseta", etc.) y pueden contener variaciones ortográficas que afectan la comparación entre cadenas.

# 2. Vectorización

In [49]:
# Vectorización TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df_clean['clean_title'])

In [58]:
title = next(c for c in df_clean.columns if 'ITE_ITEM_TITLE' in c.upper())
k = 6

nn = NearestNeighbors(metric='cosine', algorithm='brute')
nn.fit(tfidf_matrix)

# distances: menor = más similar (porque es distancia coseno)
distances, indices = nn.kneighbors(tfidf_matrix, n_neighbors=k)

# Construimos pares únicos (i < j) con score de similitud (1 - distancia)
result = []
for i in range(indices.shape[0]):
    for j_idx, d in zip(indices[i, 1:], distances[i, 1:]):  # saltamos el propio (índice 0)
        j = int(j_idx)
        if i < j:  # evitar duplicados (i,j) y (j,i)
            sim = 1.0 - float(d)
            result.append((i, j, sim))

# Crear DataFrame con índices y score
pairs_df = pd.DataFrame(pairs, columns=['idx_a', 'idx_b', 'score'])

# Duplicamos la columna original del dataset para hacer las comparaciones 
pairs_df['title_a'] = df_clean.iloc[pairs_df['idx_a']][title].to_numpy()
pairs_df['title_b'] = df_clean.iloc[pairs_df['idx_b']][title].to_numpy()

# Reordenar columnas: solo títulos y score
resultado = pairs_df[['title_a', 'title_b', 'score']].sort_values(by='score', ascending=True).reset_index(drop=True)

resultado.head()


,title_a,title_b,score
0,Ês,Tênis Feminino Olympikus - Eclipse,0.0
1,Kona Stink.....downhil,Tênis Olympikus Fibra Original,0.0
2,Piso,Tênis Feminino Olympikus - Eclipse,0.0
3,Kona Stink.....downhil,Tênis Feminino Olympikus - Eclipse,0.0
4,Kona Stink.....downhil,Tênis Casual Doma P/ Dia A Dia Promoção,0.0


Usamos NearestNeighbors porque el dataset es grande y no es práctico calcular la matriz completa de similitud. Nos enfocamos solo en los productos más parecidos a cada ítem.

In [59]:
# Generar el DataFrame de mayor a menor en similaridad
resultado = pairs_df[['title_a', 'title_b', 'score']].sort_values(by='score', ascending=False).reset_index(drop=True)

resultado.head(40)

,title_a,title_b,score
0,Tênis Sapatênis Masculino Casual,Sapatênis Tênis Masculino Casual,1.000000
1,Tenis Feminino Esportivo Actvitta Original Abs...,Tenis Feminino Esportivo Actvitta Absorção Imp...,1.000000
2,"Tênis Mvp Cross Training - Cross, Academia, Fu...","Tênis Mvp Cross Training - Cross, Academia E F...",1.000000
3,Tênis Camuflado Preto Esportivo Masculino Actv...,Tênis Esportivo Masculino Camuflado Preto Actv...,1.000000
4,Tênis On Running Cloud Preto/ Branco Masculino,Tênis On Running Cloud Masculino Preto E Branco,1.000000
5,Tênis Olympikus Masculino Contrast,Tênis Masculino Olympikus Contrast,1.000000
6,Tênis Freeday Flip Eco Todo Preto,Tênis Todo Preto Freeday Flip Eco,1.000000
7,Tênis Branco Feminino Confortável,Tênis Feminino Branco Confortável,1.000000
8,Nike Air Force I Infantil,Nike Air Force 1 Infantil,1.000000
9,Tênis Converse All Star Cano Alto,Tênis Converse All Star Cano Alto,1.000000


Este es el resultado despues de usar TF-IDF + similitud coseno sobre los títulos. El método genera pares de productos con un score entre 0 y 1.

Aunque TF-IDF funciona bien para palabras exactas, no captura sinónimos; como mejora futura se podrían usar embeddings semánticos para obtener resultados más robustos.